# Task for Today  

***

## Solar Radiation Regression  

Given *solar data from different time periods*, let's try to predict the **solar radiation** of a given period.  
  
We will use XGBoost to make our predictions.

# Getting Started

In [1]:
import numpy as np
import pandas as pd

import re
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import optuna
import xgboost as xgb

from sklearn.metrics import r2_score

c:\Users\Elnour Tech\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('SolarPrediction.csv')

In [3]:
data

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 0:00,23:55:26,1.21,48,30.46,59,177.39,5.62,6:13:00,18:13:00
1,1475229023,9/29/2016 0:00,23:50:23,1.21,48,30.46,58,176.78,3.37,6:13:00,18:13:00
2,1475228726,9/29/2016 0:00,23:45:26,1.23,48,30.46,57,158.75,3.37,6:13:00,18:13:00
3,1475228421,9/29/2016 0:00,23:40:21,1.21,48,30.46,60,137.71,3.37,6:13:00,18:13:00
4,1475228124,9/29/2016 0:00,23:35:24,1.17,48,30.46,62,104.95,5.62,6:13:00,18:13:00
...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,12/1/2016 0:00,0:20:04,1.22,44,30.43,102,145.42,6.75,6:41:00,17:42:00
32682,1480587301,12/1/2016 0:00,0:15:01,1.17,44,30.42,102,117.78,6.75,6:41:00,17:42:00
32683,1480587001,12/1/2016 0:00,0:10:01,1.20,44,30.42,102,145.19,9.00,6:41:00,17:42:00
32684,1480586702,12/1/2016 0:00,0:05:02,1.23,44,30.42,101,164.19,7.87,6:41:00,17:42:00


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32686 entries, 0 to 32685
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   UNIXTime                32686 non-null  int64  
 1   Data                    32686 non-null  object 
 2   Time                    32686 non-null  object 
 3   Radiation               32686 non-null  float64
 4   Temperature             32686 non-null  int64  
 5   Pressure                32686 non-null  float64
 6   Humidity                32686 non-null  int64  
 7   WindDirection(Degrees)  32686 non-null  float64
 8   Speed                   32686 non-null  float64
 9   TimeSunRise             32686 non-null  object 
 10  TimeSunSet              32686 non-null  object 
dtypes: float64(4), int64(3), object(4)
memory usage: 2.7+ MB


In [5]:
print("Total missing values:", data.isna().sum().sum())

Total missing values: 0


# Feature Engineering

In [6]:
data

,UNIXTime,Data,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet
0,1475229326,9/29/2016 0:00,23:55:26,1.21,48,30.46,59,177.39,5.62,6:13:00,18:13:00
1,1475229023,9/29/2016 0:00,23:50:23,1.21,48,30.46,58,176.78,3.37,6:13:00,18:13:00
2,1475228726,9/29/2016 0:00,23:45:26,1.23,48,30.46,57,158.75,3.37,6:13:00,18:13:00
3,1475228421,9/29/2016 0:00,23:40:21,1.21,48,30.46,60,137.71,3.37,6:13:00,18:13:00
4,1475228124,9/29/2016 0:00,23:35:24,1.17,48,30.46,62,104.95,5.62,6:13:00,18:13:00
...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,12/1/2016 0:00,0:20:04,1.22,44,30.43,102,145.42,6.75,6:41:00,17:42:00
32682,1480587301,12/1/2016 0:00,0:15:01,1.17,44,30.42,102,117.78,6.75,6:41:00,17:42:00
32683,1480587001,12/1/2016 0:00,0:10:01,1.20,44,30.42,102,145.19,9.00,6:41:00,17:42:00
32684,1480586702,12/1/2016 0:00,0:05:02,1.23,44,30.42,101,164.19,7.87,6:41:00,17:42:00


In [7]:
data['Month'] = data['Data'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(np.int)
data['Day'] = data['Data'].apply(lambda x: re.search(r'(?<=\/)\d+(?=\/)', x).group(0)).astype(np.int)
data['Year'] = data['Data'].apply(lambda x: re.search(r'(?<=\/)\d+(?=\s)', x).group(0)).astype(np.int)

data = data.drop('Data', axis=1)

C:\Users\Elnour Tech\AppData\Local\Temp\ipykernel_10752\2570297093.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data['Month'] = data['Data'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(np.int)
C:\Users\Elnour Tech\AppData\Local\Temp\ipykernel_10752\2570297093.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the preci

In [8]:
data

,UNIXTime,Time,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet,Month,Day,Year
0,1475229326,23:55:26,1.21,48,30.46,59,177.39,5.62,6:13:00,18:13:00,9,29,2016
1,1475229023,23:50:23,1.21,48,30.46,58,176.78,3.37,6:13:00,18:13:00,9,29,2016
2,1475228726,23:45:26,1.23,48,30.46,57,158.75,3.37,6:13:00,18:13:00,9,29,2016
3,1475228421,23:40:21,1.21,48,30.46,60,137.71,3.37,6:13:00,18:13:00,9,29,2016
4,1475228124,23:35:24,1.17,48,30.46,62,104.95,5.62,6:13:00,18:13:00,9,29,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,0:20:04,1.22,44,30.43,102,145.42,6.75,6:41:00,17:42:00,12,1,2016
32682,1480587301,0:15:01,1.17,44,30.42,102,117.78,6.75,6:41:00,17:42:00,12,1,2016
32683,1480587001,0:10:01,1.20,44,30.42,102,145.19,9.00,6:41:00,17:42:00,12,1,2016
32684,1480586702,0:05:02,1.23,44,30.42,101,164.19,7.87,6:41:00,17:42:00,12,1,2016


In [9]:
data['Hour'] = data['Time'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(np.int)
data['Minute'] = data['Time'].apply(lambda x: re.search(r'(?<=:)\d+(?=:)', x).group(0)).astype(np.int)
data['Second'] = data['Time'].apply(lambda x: re.search(r'\d+$', x).group(0)).astype(np.int)

data = data.drop('Time', axis=1)

C:\Users\Elnour Tech\AppData\Local\Temp\ipykernel_10752\4053352762.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data['Hour'] = data['Time'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(np.int)
C:\Users\Elnour Tech\AppData\Local\Temp\ipykernel_10752\4053352762.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precis

In [10]:
data

,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,TimeSunRise,TimeSunSet,Month,Day,Year,Hour,Minute,Second
0,1475229326,1.21,48,30.46,59,177.39,5.62,6:13:00,18:13:00,9,29,2016,23,55,26
1,1475229023,1.21,48,30.46,58,176.78,3.37,6:13:00,18:13:00,9,29,2016,23,50,23
2,1475228726,1.23,48,30.46,57,158.75,3.37,6:13:00,18:13:00,9,29,2016,23,45,26
3,1475228421,1.21,48,30.46,60,137.71,3.37,6:13:00,18:13:00,9,29,2016,23,40,21
4,1475228124,1.17,48,30.46,62,104.95,5.62,6:13:00,18:13:00,9,29,2016,23,35,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,1.22,44,30.43,102,145.42,6.75,6:41:00,17:42:00,12,1,2016,0,20,4
32682,1480587301,1.17,44,30.42,102,117.78,6.75,6:41:00,17:42:00,12,1,2016,0,15,1
32683,1480587001,1.20,44,30.42,102,145.19,9.00,6:41:00,17:42:00,12,1,2016,0,10,1
32684,1480586702,1.23,44,30.42,101,164.19,7.87,6:41:00,17:42:00,12,1,2016,0,5,2


In [11]:
data['SunriseHour'] = data['TimeSunRise'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(np.int)
data['SunriseMinute'] = data['TimeSunRise'].apply(lambda x: re.search(r'(?<=:)\d+(?=:)', x).group(0)).astype(np.int)

data['SunsetHour'] = data['TimeSunSet'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(np.int)
data['SunsetMinute'] = data['TimeSunSet'].apply(lambda x: re.search(r'(?<=:)\d+(?=:)', x).group(0)).astype(np.int)

data = data.drop(['TimeSunRise', 'TimeSunSet'], axis=1)

C:\Users\Elnour Tech\AppData\Local\Temp\ipykernel_10752\755505004.py:1: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  data['SunriseHour'] = data['TimeSunRise'].apply(lambda x: re.search(r'^\d+', x).group(0)).astype(np.int)
C:\Users\Elnour Tech\AppData\Local\Temp\ipykernel_10752\755505004.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specif

In [12]:
data

,UNIXTime,Radiation,Temperature,Pressure,Humidity,WindDirection(Degrees),Speed,Month,Day,Year,Hour,Minute,Second,SunriseHour,SunriseMinute,SunsetHour,SunsetMinute
0,1475229326,1.21,48,30.46,59,177.39,5.62,9,29,2016,23,55,26,6,13,18,13
1,1475229023,1.21,48,30.46,58,176.78,3.37,9,29,2016,23,50,23,6,13,18,13
2,1475228726,1.23,48,30.46,57,158.75,3.37,9,29,2016,23,45,26,6,13,18,13
3,1475228421,1.21,48,30.46,60,137.71,3.37,9,29,2016,23,40,21,6,13,18,13
4,1475228124,1.17,48,30.46,62,104.95,5.62,9,29,2016,23,35,24,6,13,18,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32681,1480587604,1.22,44,30.43,102,145.42,6.75,12,1,2016,0,20,4,6,41,17,42
32682,1480587301,1.17,44,30.42,102,117.78,6.75,12,1,2016,0,15,1,6,41,17,42
32683,1480587001,1.20,44,30.42,102,145.19,9.00,12,1,2016,0,10,1,6,41,17,42
32684,1480586702,1.23,44,30.42,101,164.19,7.87,12,1,2016,0,5,2,6,41,17,42


In [13]:
data.dtypes

UNIXTime                    int64
Radiation                 float64
Temperature                 int64
Pressure                  float64
Humidity                    int64
WindDirection(Degrees)    float64
Speed                     float64
Month                       int32
Day                         int32
Year                        int32
Hour                        int32
Minute                      int32
Second                      int32
SunriseHour                 int32
SunriseMinute               int32
SunsetHour                  int32
SunsetMinute                int32
dtype: object

In [14]:
data['Year'].unique()

array([2016])

In [15]:
data['SunriseHour'].unique()

array([6])

In [16]:
data = data.drop(['Year', 'SunriseHour'], axis=1)

# Splitting/Scaling

In [17]:
y = data['Radiation'].copy()
X = data.drop('Radiation', axis=1).copy()

In [18]:
scaler = StandardScaler()

X = scaler.fit_transform(X)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=100)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, train_size=0.8, random_state=200)

In [20]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)
dtest = xgb.DMatrix(X_test, label=y_test)

# Hyperparameter Search

In [21]:
def get_model_rmse(params):
    model = xgb.train(params, dtrain, num_boost_round=100, evals=[(dval, 'eval')], early_stopping_rounds=10, verbose_eval=0)
    results = model.eval(dval)
    rmse = np.float(re.search(r'[\d.]+$', results).group(0))
    return rmse

In [22]:
def objective(trial):
    learning_rate = trial.suggest_loguniform('learning_rate', 0.00001, 10.0)
    max_depth = trial.suggest_int('max_depth', 4, 8)
    l1_reg = trial.suggest_loguniform('l1_reg', 0.00001, 10.0)
    l2_reg = trial.suggest_loguniform('l2_reg', 0.00001, 10.0)
    
    params = {'learning_rate': learning_rate, 'max_depth': max_depth, 'alpha': l1_reg, 'lambda': l2_reg}
    
    return get_model_rmse(params)

In [23]:
study = optuna.create_study()
study.optimize(objective, n_trials=100, show_progress_bar=True)

[I 2023-03-10 17:12:14,698] A new study created in memory with name: no-name-84f42043-cce1-4fcb-945c-b3c4d376112a
c:\Users\Elnour Tech\AppData\Local\Programs\Python\Python311\Lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()
  0%|          | 0/100 [00:00<?, ?it/s]C:\Users\Elnour Tech\AppData\Local\Temp\ipykernel_10752\1559402670.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rmse = np.float(re.search(r'[\d.]+$', results).group(0))
  1%|          | 1/100 [00:12<20:38, 12.51s/it]

[I 2023-03-10 17:12:27,276] Trial 0 finished with value: 337.28957016760427 and parameters: {'learning_rate': 0.0014065160690982215, 'max_depth': 7, 'l1_reg': 0.09869722472102854, 'l2_reg': 3.789369096592616}. Best is trial 0 with value: 337.28957016760427.


C:\Users\Elnour Tech\AppData\Local\Temp\ipykernel_10752\1559402670.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rmse = np.float(re.search(r'[\d.]+$', results).group(0))
  2%|▏         | 2/100 [00:15<11:25,  6.99s/it]

[I 2023-03-10 17:12:30,400] Trial 1 finished with value: 77938887806.7053 and parameters: {'learning_rate': 6.7188753454362065, 'max_depth': 8, 'l1_reg': 3.1903118167777884, 'l2_reg': 6.256527055110673e-05}. Best is trial 0 with value: 337.28957016760427.


C:\Users\Elnour Tech\AppData\Local\Temp\ipykernel_10752\1559402670.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rmse = np.float(re.search(r'[\d.]+$', results).group(0))
  3%|▎         | 3/100 [00:29<16:35, 10.26s/it]

...

[I 2023-03-10 17:20:49,097] Trial 97 finished with value: 84.94147650049285 and parameters: {'learning_rate': 0.24915454218344188, 'max_depth': 8, 'l1_reg': 4.394955641784536, 'l2_reg': 0.8915257708535833}. Best is trial 82 with value: 82.2347423018513.


C:\Users\Elnour Tech\AppData\Local\Temp\ipykernel_10752\1559402670.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rmse = np.float(re.search(r'[\d.]+$', results).group(0))
 99%|█████████▉| 99/100 [08:37<00:03,  3.61s/it]

[I 2023-03-10 17:20:52,126] Trial 98 finished with value: 87.27934109940004 and parameters: {'learning_rate': 0.3748101344856861, 'max_depth': 8, 'l1_reg': 0.016329318398173455, 'l2_reg': 4.4048867138027274}. Best is trial 82 with value: 82.2347423018513.


C:\Users\Elnour Tech\AppData\Local\Temp\ipykernel_10752\1559402670.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  rmse = np.float(re.search(r'[\d.]+$', results).group(0))
100%|██████████| 100/100 [08:41<00:00,  5.21s/it]

[I 2023-03-10 17:20:56,039] Trial 99 finished with value: 85.67468014016085 and parameters: {'learning_rate': 0.12555804009514426, 'max_depth': 7, 'l1_reg': 0.07107233506030511, 'l2_reg': 2.8130778801667287}. Best is trial 82 with value: 82.2347423018513.


In [24]:
best_params = study.best_params
best_params

{'learning_rate': 0.1703224493504231,
 'max_depth': 8,
 'l1_reg': 0.22095308386241375,
 'l2_reg': 3.972236428843102}

In [25]:
model = xgb.train(best_params, dtrain, num_boost_round=10000, evals=[(dval, 'eval')], early_stopping_rounds=10)

[17:21:22] WARNING: C:/buildkite-agent/builds/buildkite-windows-cpu-autoscaling-group-i-030221e36e1a46bfb-1/xgboost/xgboost-ci-windows/src/learner.cc:767: 
Parameters: { "l1_reg", "l2_reg" } are not used.

[0]	eval-rmse:322.89721
[1]	eval-rmse:274.94966
[2]	eval-rmse:235.78431
[3]	eval-rmse:204.18025
[4]	eval-rmse:178.88136
[5]	eval-rmse:158.51222
[6]	eval-rmse:142.44514
[7]	eval-rmse:129.58083
[8]	eval-rmse:119.52673
[9]	eval-rmse:111.97305
[10]	eval-rmse:106.26452
[11]	eval-rmse:102.17033
[12]	eval-rmse:98.75725
[13]	eval-rmse:95.92130
[14]	eval-rmse:93.61825
[15]	eval-rmse:92.49343
[16]	eval-rmse:90.76101
[17]	eval-rmse:90.03426
[18]	eval-rmse:89.24603
[19]	eval-rmse:88.63354
[20]	eval-rmse:87.97522
[21]	eval-rmse:87.61389
[22]	eval-rmse:87.27503
[23]	eval-rmse:87.16988
[24]	eval-rmse:87.05573
[25]	eval-rmse:86.64969
[26]	eval-rmse:86.53250
[27]	eval-rmse:86.53530
[28]	eval-rmse:86.50787
[29]	eval-rmse:86.48370
[30]	eval-rmse:86.41542
[31]	eval-rmse:86.09194
[32]	eval-rmse:85.93675


In [26]:
import joblib 
filename = "complite_model.joblib"
joblib.dump(model,filename)

['complite_model.joblib']

# Results

In [27]:
y_true = np.array(y_test, dtype=np.float)
y_pred = np.array(model.predict(dtest), dtype=np.float)

C:\Users\Elnour Tech\AppData\Local\Temp\ipykernel_10752\2899363029.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_true = np.array(y_test, dtype=np.float)
C:\Users\Elnour Tech\AppData\Local\Temp\ipykernel_10752\2899363029.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_pred = np.array(model.predict(dtest), dtype=np.float)


In [28]:
r2 = r2_score(y_true, y_pred)

print("R^2 Score: {:.4f}".format(r2))

R^2 Score: 0.9401


In [32]:
for i in y_pred:
    print (i)

print(len(y))

-0.00474175438284874
3.3925058841705322
0.5694091320037842
259.8631591796875
176.57913208007812
-0.07314882427453995
0.6222001314163208
741.5903930664062
-1.7303088903427124
2.2974164485931396
860.7632446289062
2.9853029251098633
0.13838934898376465
987.297119140625
974.6217651367188
4.829600811004639
290.2562561035156
521.2321166992188
889.309326171875
2.405935287475586
0.17069311439990997
512.97705078125
1.6578330993652344
1.5158618688583374
428.3459777832031
4.009169101715088
-2.8239145278930664
64.510498046875
-0.3580481708049774
-3.6327390670776367
3.521038055419922
-3.4792442321777344
631.3421630859375
806.9871215820312
2.600641965866089
404.6895446777344
347.67852783203125
284.9770812988281
1.6142336130142212
3.558809757232666
955.8170166015625
1.7702596187591553
3.0654349327087402
34.85417175292969
1.8078473806381226
1.355758786201477
1.6809914112091064
29.7291259765625
-3.0868794918060303
9.878746032714844
-0.24693989753723145
3.5381829738616943
20.16649627685547
730.278259277